In [1]:
from src import *

## Description



GOAL: 

## Table of Contents
- [Standard Sliding window Dynamic (Visualization)](#sliding)

- [Standard Sliding window Dynamic (Analysis)](#slidinganalysis)

In [2]:
mri_path = "./data/fmri_compile.csv"
mri_df   = pd.read_csv(mri_path)

### Standard Sliding window Dynamic
We don't know the TR but expect it to be around 1 sec and so we have 1 min to be 60 sample points. As advised (at least 1 min) for window size we pick 60 frames.

<a class="anchor" id="sliding"></a>
### Visualization

In [3]:
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')

filename_movie = 'TC_400_sub-S09_ses-4_Sintel.csv'
filename_rest  = 'TC_400_sub-S23_ses-1_Rest.csv'

movie_series, movie_df = df_to_timeseries(mri_df, filename_movie)
rest_series, rest_df   = df_to_timeseries(mri_df, filename_rest)

In [25]:
# variable for dFC
ws   = 300
step = 1 # let's overlap pretty much all for now to see smoothness

Computing Dynamic FC for rest

In [26]:
nbr, nbt = rest_series.T.shape

dFC_rest = np.zeros((nbt // step + 1 - ws, nbr,nbr))

for c, sidx in enumerate(range(0, nbt, step)):
    T = rest_series[sidx:sidx+ws]
    if T.shape[0] != ws: 
        continue
    correlation_measure = ConnectivityMeasure(kind='correlation')
    dFC_rest[c] = correlation_measure.fit_transform([T])[0]

correlation_measure = ConnectivityMeasure(kind='correlation')
ref_FC_rest = correlation_measure.fit_transform([rest_series])[0]

Computing Dynamic FC for movie

In [27]:
nbr, nbt = movie_series.T.shape

dFC_movie = np.zeros((nbt // step + 1 - ws, nbr,nbr))

for c, sidx in enumerate(range(0, nbt, step)):
    T = movie_series[sidx:sidx+ws]
    if T.shape[0] != ws: 
        continue
    correlation_measure = ConnectivityMeasure(kind='correlation')
    dFC_movie[c] = correlation_measure.fit_transform([T])[0]

correlation_measure = ConnectivityMeasure(kind='correlation')
ref_FC_movie = correlation_measure.fit_transform([movie_series])[0]

In [28]:
from brainspace.gradient import GradientMaps

embedding  = "dm" # diffusion map
aff_kernel = "pearson" #affinity matrix kernel
align_meth = "procrustes"
nb_comp    = 2

gref_rest = GradientMaps(n_components=nb_comp, approach=embedding, kernel=aff_kernel)
gref_rest.fit(ref_FC_rest)

gref_movie = GradientMaps(n_components=nb_comp, approach=embedding, kernel=aff_kernel)
gref_movie.fit(ref_FC_movie)

# Computing Rest Gradients
G_rest  = []
for widx in tqdm(range(0, len(dFC_rest))):

    galign = GradientMaps(n_components=nb_comp, kernel=aff_kernel, approach=embedding, alignment=align_meth)
    galign.fit(dFC_rest[widx], reference=gref_rest.gradients_)
    G_rest.append(galign.gradients_)

# Computing Movie Gradients
G_movie = []
for widx in tqdm(range(0, len(dFC_movie))):

    galign = GradientMaps(n_components=nb_comp, kernel=aff_kernel, approach=embedding, alignment=align_meth)
    galign.fit(dFC_movie[widx], reference=gref_movie.gradients_)
    G_movie.append(galign.gradients_)

G_rest  = np.asarray(G_rest)
G_movie = np.asarray(G_movie)

100%|██████████| 412/412 [00:07<00:00, 54.33it/s]


In [29]:
partial_region_movie = { k: movie_df[movie_df.vindex == k]['partial_region'].iloc[0]
                    for k in movie_df['vindex'].unique()}
partial_region_rest  = { k: rest_df[rest_df.vindex == k]['partial_region'].iloc[0]
                    for k in rest_df['vindex'].unique()}

In [37]:
import seaborn as sns
coloring  = [partial_region_movie[i] for i in range(400)]
ts = ['rest', 'movie']

gs = [G_rest, G_movie]

for i in range(2):
    for j in tqdm(range(len(gs[i]))):
        tmp_df = {"G1":gs[i][j][:,0] , "G2": gs[i][j][:,1], "region": coloring}
        ax     = sns.jointplot(data=tmp_df, x="G1", y="G2", 
                    hue="region", height=5, 
                    xlim=(gs[i][:,0].min()*1.5,gs[i][:,0].max()*1.5), 
                    ylim=(gs[i][:,1].min()*1.5,gs[i][:,1].max()*1.5))
        ax.fig.suptitle("Gradients's closenedness plot ({})".format(ts[i]))
        legend_properties = {'weight':'bold','size':6}
        ax.ax_joint.legend(prop=legend_properties,loc='upper right')
        ax.savefig("./media/gradient_plots/{}_{}.jpg".format(ts[i],j))

        #this is the line to be added to avoid showing all the list of plots
        plt.close("all") 


100%|██████████| 412/412 [02:27<00:00,  2.79it/s]


In [39]:
# path = "{}/master_project/brainspace_demo/media/gradient_plots".format(ROOT)

# # reorder saved images path
# rest_path  = loadimg_in_order([r for r in os.listdir(path) if 'rest' in r])
# movie_path = loadimg_in_order([m for m in os.listdir(path) if 'movie' in m])

# rest_array  = [cv2.imread(path+'/'+filename)[:,:,::-1] for filename in rest_path]
# movie_array = [cv2.imread(path+'/'+filename)[:,:,::-1] for filename in movie_path]

# img2video(rest_array, 3, outpath_name='./media/rest_plots.mp4')
# img2video(movie_array, 3, outpath_name='./media/movie_plots.mp4')

<a class="anchor" id="slidinganalysis"></a>
### Analysis

In [48]:
gs = [G_rest, G_movie]
tmp_df = pd.DataFrame.from_dict(tmp_df) # defined from previous plot

In [53]:
tmp_df.region.unique()

array(['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 'Limbic', 'Cont',
       'Default'], dtype=object)

In [51]:
list(tmp_df[tmp_df.region=='Vis'].index)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
             13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
             26,  27,  28,  29,  30, 200, 201, 202, 203, 204, 205, 206, 207,
            208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,
            221, 222, 223, 224, 225, 226, 227, 228, 229],
           dtype='int64')

In [44]:
G_rest[:,:,0]

array([[ 9.24564695, 11.07860739,  4.11536152, ...,  4.04292376,
        10.03191294,  8.840907  ],
       [10.977097  , 12.53246023,  4.92668605, ...,  3.89637712,
        11.05397252,  9.75068104],
       [11.51861767, 12.91690661,  4.79440588, ...,  4.22275815,
        11.15807691,  9.88467771],
       ...,
       [10.08068345,  9.88693506,  7.72073112, ..., -0.93709564,
         7.61615117,  6.9592662 ],
       [10.47864607, 10.13122946,  8.18095511, ..., -1.5393993 ,
         7.53040754,  6.90201834],
       [10.57631356, 10.05065518,  8.00377123, ..., -1.04507373,
         7.57874089,  6.93131907]])

In [43]:
G_rest.shape

(161, 400, 2)